In [1]:
from sklearn.model_selection import StratifiedKFold, train_test_split
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from xgboost import XGBClassifier
import pickle
from sklearn.base import clone
from sklearn.metrics import matthews_corrcoef
from xgboost import XGBClassifier
from sklearn.metrics import matthews_corrcoef

from sklearn.model_selection import StratifiedKFold
import numpy as np
import optuna
from optuna.samplers import TPESampler


In [2]:
test_data =pd.read_csv('test_proc.csv')

In [3]:
data =pd.read_csv('transformed.csv')
data.set_index('id',inplace=True)
data =data.iloc[:,1:]
X = data.drop(columns=['class'],axis=1)

In [4]:
y = data['class']
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
xgboost_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42,n_estimators=1000, 
                        max_depth=5,tree_method= 'gpu_hist')

In [ ]:
def objective(trial):
    # Define hyperparameters to tune
    params = {
        'max_depth': trial.suggest_int('max_depth', 2, 10),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 1.0, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'tree_method': 'gpu_hist'
    }
    xgboost_model = XGBClassifier(**params,use_label_encoder=False, eval_metric='logloss',andom_state=42,n_estimators=1000)
    # Train XGBoost model with the selected hyperparameters
    
    xgboost_model.fit(X_train, y_train)

    # Evaluate model on validation set
    val_score = xgboost_model.score(X_test, y_test)
    return val_score

In [ ]:
study = optuna.create_study(direction='maximize', sampler=TPESampler(seed=42))
study.optimize(objective, n_trials=10)

In [ ]:
study.best_params

In [6]:
model = XGBClassifier(use_label_encoder=False,max_depth= 7,
 learning_rate = 0.13311216080736885,
 subsample = 0.5102922471479012,
 colsample_bytree = 0.9849549260809971,tree_method= 'gpu_hist')

In [21]:
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

def k_fold_model_trainer(model, X, y, skf, test_data):
    """
    Trains the model using Stratified K-Fold cross-validation and evaluates using MCC.
    Returns the average validation MCC and test predictions.
    """
    train_mcc_scores = []
    val_mcc_scores = []
    test_preds = np.zeros(len(test_data))
    fold_number = 1
    for train_index, val_index in skf.split(X, y):
        print(f'Fold {fold_number}')
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y[train_index], y[val_index]
        if fold_number == 1:
            oof_xgb = np.zeros(len(X_val))
        # Clone the model to ensure fresh parameters
        model_clone = clone(model)
        
        # Fit the model
        model_clone.fit(X_train, y_train)
        
        # Predict on training and validation sets
        y_train_pred = model_clone.predict(X_train)
        y_val_pred = model_clone.predict(X_val)
        
        # Calculate MCC scores
        train_mcc = matthews_corrcoef(y_train, y_train_pred)
        val_mcc = matthews_corrcoef(y_val, y_val_pred)
        
        print(f'Training MCC: {train_mcc:.4f}')
        print(f'Validation MCC: {val_mcc:.4f}\n')
        
        train_mcc_scores.append(train_mcc)
        val_mcc_scores.append(val_mcc)
        oof_xgb += model_clone.predict_proba(X_val)[:, 1]
        # Predict on test data
        test_preds += model_clone.predict_proba(test_data)[:, 1]
        fold_number += 1
    
    # Average test predictions
    oof_xgb /= n_splits
    test_preds /= n_splits
    
    # Average MCC scores
    avg_train_mcc = np.mean(train_mcc_scores)
    avg_val_mcc = np.mean(val_mcc_scores)
    
    print(f'Average Training MCC: {avg_train_mcc:.4f}')
    print(f'Average Validation MCC: {avg_val_mcc:.4f}')
    
    return avg_val_mcc, test_preds,oof_xgb

In [22]:
avg_val_mcc,test_preds,oof_xgb = k_fold_model_trainer(model, X, y, skf, test_data)

Fold 1


c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\core.py:158: UserWarning: [23:35:37] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\core.py:158: UserWarning: [23:35:37] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\core.py:158: UserWarning: [23:35:39] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-aut

Training MCC: 0.9793
Validation MCC: 0.9790

Fold 2


c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\core.py:158: UserWarning: [23:35:42] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\core.py:158: UserWarning: [23:35:42] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\core.py:158: UserWarning: [23:35:43] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-aut

Training MCC: 0.9793
Validation MCC: 0.9790

Fold 3


...

Training MCC: 0.9797
Validation MCC: 0.9798

Fold 4


c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\core.py:158: UserWarning: [23:35:51] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\core.py:158: UserWarning: [23:35:51] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\core.py:158: UserWarning: [23:35:53] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-aut

Training MCC: 0.9805
Validation MCC: 0.9797

Fold 5


c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\core.py:158: UserWarning: [23:35:56] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\core.py:158: UserWarning: [23:35:56] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\core.py:158: UserWarning: [23:35:57] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-aut

Training MCC: 0.9802
Validation MCC: 0.9799

Average Training MCC: 0.9798
Average Validation MCC: 0.9795


In [23]:
oof_xgb

array([0.22301083, 0.4017913 , 0.60770768, ..., 0.59392647, 0.42690391,
       0.60550247])

In [25]:
oof_cat =np.load('val_preds_cat.npy')[:,1]

In [26]:
oof_cat

array([0.00198086, 0.99837444, 0.07073228, ..., 0.9987155 , 0.99815132,
       0.99948941])

In [27]:
pip install hillclimbers

Using legacy 'setup.py install' for hillclimbers, since package 'wheel' is not installed.
    Running setup.py install for hillclimbers: started
    Running setup.py install for hillclimbers: finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [29]:
from hillclimbers import climb_hill, partial

In [ ]:
test_preds = climb_hill( 
                 oof_pred_df=pd.concat(oof_cat,oof_xgb), 
                 test_pred_df=test_pred_df,
                 target="yield",
                 objective="minimize", 
                 eval_metric=partial(mean_absolute_error),
                 negative_weights=False, 
                 precision=0.001, 
                 plot_hill=True, 
                 plot_hist=True
            )
